In [2]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pickle

# Définir l'URI de suivi MLflow
mlflow_tracking_uri = "http://127.0.0.1:5000"
experiment_name = "xgboost_salary_prediction"

# Configurer l'URI de suivi MLflow
mlflow.set_tracking_uri(mlflow_tracking_uri)
client = MlflowClient(tracking_uri=mlflow_tracking_uri)

# Obtenir l'expérience MLflow
experiment = client.get_experiment_by_name(experiment_name)

# Rechercher tous les runs de l'expérience
runs = client.search_runs(experiment_ids=[experiment.experiment_id])

# Trouver le run avec le meilleur R2
best_run = None
best_r2 = float('-inf')

for run in runs:
    # Vérifiez si la métrique r2_test est présente dans le run
    if 'r2_test' in run.data.metrics:
        r2_test = run.data.metrics['r2_test']
        if r2_test > best_r2:
            best_r2 = r2_test
            best_run = run

if best_run:
    print(f"Best run ID: {best_run.info.run_id}, R2: {best_r2}")

    # Télécharger le meilleur modèle
    model_uri = f"runs:/{best_run.info.run_id}/xgboost_model"
    best_model = mlflow.sklearn.load_model(model_uri)

    # Enregistrer le modèle localement dans un fichier .pkl
    with open("../api/best_xgboost_model.pkl", "wb") as model_file:
        pickle.dump(best_model, model_file)
    print("Best model saved locally as 'best_xgboost_model.pkl'")
else:
    print("No valid runs found.")


Best run ID: 0c2002b75a6b4660b4f3f352028db8ad, R2: 0.31635087055046396


Best model saved locally as 'best_xgboost_model.pkl'
